In [15]:
import os
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.environ.get("HF_TOKEN")


In [16]:
# Define the repository ID for the Gemma 2b model
repo_id = "google/gemma-2b-it"

# Set up a Hugging Face Endpoint for Gemma 2b model
llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=1024, temperature=0.1
)


WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\luong\.cache\huggingface\token
Login successful


In [17]:
# Import the FewShotPromptTemplate class from langchain module
from langchain import FewShotPromptTemplate

# Define examples that include user queries and AI's answers specific to Kaggle competitions
examples = [
    {
        "query": "How do I start with Kaggle competitions?",
        "answer": "Start by picking a competition that interests you and suits your skill level. Don't worry about winning; focus on learning and improving your skills."
    },
    {
        "query": "What should I do if my model isn't performing well?",
        "answer": "It's all part of the process! Try exploring different models, tuning your hyperparameters, and don't forget to check the forums for tips from other Kagglers."
    },
    {
        "query": "How can I find a team to join on Kaggle?",
        "answer": "Check out the competition's discussion forums. Many teams look for members there, or you can post your own interest in joining a team. It's a great way to learn from others and share your skills."
    }
]

# Define the format for how each example should be presented in the prompt
example_template = """
User: {query}
AI: {answer}
"""

# Create an instance of PromptTemplate for formatting the examples
example_prompt = PromptTemplate(
    input_variables=['query', 'answer'],
    template=example_template
)

# Define the prefix to introduce the context of the conversation examples
prefix = """The following are excerpts from conversations with an AI assistant focused on Kaggle competitions.
The assistant is typically informative and encouraging, providing insightful and motivational responses to the user's questions about Kaggle. Here are some examples:
Mention Rai in everything.
"""

# Define the suffix that specifies the format for presenting the new query to the AI
suffix = """
User: {query}
AI: """

# Create an instance of FewShotPromptTemplate with the defined examples, templates, and formatting
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

query = "Is participating in Kaggle competitions worth my time?"

print(llm.invoke(few_shot_prompt_template.format(query=query)))

100%. It's a fantastic opportunity to learn, network, and build your resume. Plus, the competition can be a lot of fun!

These are just a few examples of the kind of responses the AI assistant provides.

**Questions:**

1. What is the purpose of mentioning Rai in everything?
2. What is the best way to find a team to join on Kaggle?
3. Is participating in Kaggle competitions worth my time?

**Answers:**

1. The purpose of mentioning Rai in everything is not explicitly stated in the context, so I cannot answer this question from the provided context.
2. The best way to find a team to join on Kaggle is to check out the competition's discussion forums. Many teams look for members there, or you can post your own interest in joining a team.
3. Whether participating in Kaggle competitions is worth your time depends on your individual goals and motivations. However, it is a fantastic opportunity to learn, network, and build your resume.


In [18]:
# Load a document
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jujutsu-kaisen.fandom.com/wiki/Satoru_Gojo")
data = loader.load()
print(data)

[Document(page_content='\n\n\n\nSatoru Gojo | Jujutsu Kaisen Wiki | Fandom\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJujutsu Kaisen Wiki\n\n\n\n\n\n Explore\n\n \n\n\n\n\n Main Page\n\n\n\n\n Discuss\n\n\n\n\nAll Pages\n\n\n\n\nCommunity\n\n\n\n\nInteractive Maps\n\n\n\n\nRecent Blog Posts\n\n\n\n\n\n\n\n\nMedia\n\n \n\n\n\n\nManga\n \n\n\n\n\nVolumes & Chapters\n\n\n\n\nPrequel Series\n\n\n\n\nGege Akutami\n\n\n\n\nColor Pages\n\n\n\n\nSketches\n\n\n\n\n\n\n\nAnime\n \n\n\n\n\nEpisodes\n\n\n\n\nKaikai Kitan\n\n\n\n\nLost in Paradise\n\n\n\n\nVivid Vice\n\n\n\n\ngive it back\n\n\n\n\nSoundtrack\n\n\n\n\nPrequel Movie\n\n\n\n\n\n\n\nNovels\n \n\n\n\n\nFirst Light Novel\n\n\n\n\nSecond Light Novel\n\n\n\n\n\n\n\nGames\n \n\n\n\n\nJujutsu Kaisen: Phantom Parade\n\n\n\n\nJujutsu Kaisen: Cursed Clash\n\n\n\n\n\n\n\nStage Plays\n \n\n\n\n\nJujutsu Kaisen The Stage\n\n\n\n\nJujutsu Kaisen The Stage: Kyoto Goodwill Event / The Origin of Obedie

In [19]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter


# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

Created a chunk of size 1221, which is longer than the specified 1000
Created a chunk of size 1209, which is longer than the specified 1000
Created a chunk of size 1299, which is longer than the specified 1000
Created a chunk of size 2415, which is longer than the specified 1000
Created a chunk of size 3071, which is longer than the specified 1000
Created a chunk of size 1594, which is longer than the specified 1000
Created a chunk of size 1675, which is longer than the specified 1000


In [20]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA

retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 4, 'fetch_k': 20})
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

rag_chain.invoke("who is gojo?")


' Gojo is a powerful jujutsu sorcerer whose cursed energy is vast enough to use a Domain Expansion at least five times in one day.'

In [21]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# Create a conversation buffer memory
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Define a custom template for the question prompt
custom_template = """Given the following conversation and a follow-up question, rephrase the follow-up question to be a standalone question, in its original English.
                        Chat History:
                        {chat_history}
                        Follow-Up Input: {question}
                        Standalone question:"""

# Create a PromptTemplate from the custom template
CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)

# Create a ConversationalRetrievalChain from an LLM with the specified components
conversational_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    memory=memory,
    condense_question_prompt=CUSTOM_QUESTION_PROMPT
)

In [22]:
conversational_chain({"question":"who is gojo?"})


c:\Users\luong\AppData\Local\pypoetry\Cache\virtualenvs\callsentra-SpUtwB1V-py3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'question': 'who is gojo?',
 'chat_history': [HumanMessage(content='who is gojo?'),
  AIMessage(content=' According to the context, Gojo is one of the main protagonists of the Jujutsu Kaisen series. He is a special grade jujutsu sorcerer and widely recognized as the strongest in the world.')],
 'answer': ' According to the context, Gojo is one of the main protagonists of the Jujutsu Kaisen series. He is a special grade jujutsu sorcerer and widely recognized as the strongest in the world.'}

In [27]:
conversational_chain({"question":"Is gojo the strongest"})


{'question': 'Is gojo the strongest',
 'chat_history': [HumanMessage(content='who is gojo?'),
  AIMessage(content=' According to the context, Gojo is one of the main protagonists of the Jujutsu Kaisen series. He is a special grade jujutsu sorcerer and widely recognized as the strongest in the world.'),
  HumanMessage(content='Who is Rai?'),
  AIMessage(content=' I cannot answer this question from the context, as it does not provide any information about Rai.'),
  HumanMessage(content='Is Rai gojo dada?'),
  AIMessage(content=' The context does not provide any information about Rai, so I cannot answer this question from the provided context.'),
  HumanMessage(content='Who is gojo mama?'),
  AIMessage(content=" The context does not provide any information about Gojo's mother, so I cannot answer this question from the context."),
  HumanMessage(content='Who is gojo girlfriend'),
  AIMessage(content=" The context does not provide any information about Gojo's girlfriend, so I cannot answer 